<a href="https://colab.research.google.com/github/eveungs/deteccionAguaTierra/blob/main/Detecci%C3%B3nAguaTierra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/05_drawing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.



---

---


                     ***APLICACIÓN PARA LA DETECCIÓN DEL LÍMITE AGUA-TIERRA CON METODOLOGÍAS SEMI-AUTOMÁTICAS***


---

---

Esta aplicación genera un producto que permite hacer una distinción clara entre el entorno marino-fluvial y terrestre. Dicha clasificación se puede obtener como un archivo raster o vectorial de una zona en particular, con la posibilidad de descarga.



Para realizar esta metodología se partió de un código en Google Earth Engine elaborado por el Departamento de Aplicaciones Geoespaciales, de la Dirección de Información Geoespacial, del Instituto Geográfico Nacional: "Detección del límite agua-tierra mediante el algoritmo mínima distancia en la nube de Google Earth Engine", que a través del uso de técnicas de teledetección, imágenes SENTINEL-2A/B MSI Nivel 1C y métodos estadísticos como distancia mínima se obtiene un clasificación de Agua/Tierra o Agua/No Agua.




---

Para hacer uso de esta aplicación se deben seguir una serie de pasos. El más importante está en el punto 3, ya que se deben elegir algunos parámetros. Luego, no es necesario hacer modificaciones en otra solapa para que funcione (a menos que se desee modificar algo del código).

Algunas cuestiones a tener en cuenta:

* Se recomienda **ejecutar el código con las solapas colapsadas**, así se asegura que se ejecutan todas las líneas necesarias.

* Las **solapas con * se deben abrir luego de ejecutarlas**, para realizar alguna acción.

* En Colab, las **sesiones son temporales** y los recursos son compartidos, por lo que si no hay interacción con el cuaderno en un cierto periódo de tiempo la sesión se desconecta y probablemente al ejecutar alguna línea de código se generará un **error**. Para solucionarlo, simplemente se deben volver a ejecutar los pasos desde el primero.

---


## 1. Instalacion de bibliotecas
Se debe **autorizar** la conexión con Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geemap
!pip install requests
!pip install pycrs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=c93757f813caec2ba9a802a9bb5301de12960028f87e17233216f1b2d7edc58c
  Stored

## 2. Instalación de módulos *
Se debe autorizar el acceso que necesita GEE. Para ello se debe ejecutar el código, y luego, **abrir la solapa** e ingresar al **URL** del segundo bloque y seguir la instrucciones. Esto genera un código de verificación que debe ser pegado a continuación.

In [ ]:
import ee
import geemap
import ipywidgets as widgets
from datetime import datetime
import pandas as pd
import requests
from IPython.display import display
import os

En el siguiente paso se deberá **ingresar al URL**, y luego, **pegar el código** generado.

In [ ]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=UJL2VjjCndwfAGomwGbZZt0QX0ENcQQvCVLZOf3QNZQ&tc=5W1eLcFJy_3bcra6GPKyv7ruZlr0bcx8XrIZEu2HcMc&cc=7KWFa8Qoe8S6ggvQco8X9MDup8gvsnStf7JqwPmxf5I

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhW7Lb1yb-jb5zJMwMBOqccg7UrlAsPlnfskwnsAv8FdIW2_aui-y-Q

Successfully saved authorization token.


## 3. Mapa base, área de estudio y período de tiempo *
Se debe ejecutar este punto y luego extender la solapa. Se cargará un mapa y debajo se desplegarán algunas opciones tales como **período de tiempo** de las imágenes, y **forma de selección del área** a clasificar.

Si se elige la opción "*Dibujar polígono en el mapa*", con las herramientas de dibujo, se debe trazar el polígono. Tener en cuenta, que producto de las limitaciones de GEE, es recomendable trabajar en un área pequeña.

Si la opción es "*Utilizar carta*", debajo se imprimirá una lista con todas las opciones de cartas 1:50.000 (a medida que se presionan se van cargando en el mapa). En este caso se debe seleccionar una para la clasificación.


---En este punto es **muy importante completar parámetros**, básicamente para que funcione la aplicación---



In [ ]:
ee.Initialize()# Inicializar Earth Engine

Map = geemap.Map(center=(-40, -64), zoom=4) #crear el mapa inetrcativo con coordenadas especificas
Map.add_basemap('HYBRID')
Map #cargar mapa

CPU times: user 455 ms, sys: 26.8 ms, total: 482 ms
Wall time: 1.32 s


Map(center=[-40, -64], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Interfaz código

In [ ]:
#..................................................... Cartas
cartas = ee.FeatureCollection("projects/e-tenglerevelyn/assets/Cartas_50000")

# Obtener los valores únicos de la columna "carac"
valoresCarac = cartas.aggregate_array("carac").distinct().getInfo()

# Crear etiquetas
etiqueta = widgets.Label(value='Seleccione una carta 1:50.000')
etiqueta_fecha = widgets.Label(value='Elija el período de las imágenes')

# Crear una lista desplegable con los registros
lista_registros = widgets.Select(options=valoresCarac, description='Opciones')


# Definir una función para manejar el evento de cambio de selección
def on_value_change(change):
    valor_carta = change['new']
    geometria_seleccionada = cartas.filter(ee.Filter.eq("carac", valor_carta))

    Map.addLayer(geometria_seleccionada, {}, 'Geometría seleccionada')


# Registrar la función de manejo del evento de cambio
lista_registros.observe(on_value_change, names='value')



#..................................................... Período de tiempo
date_picker_inicio = widgets.DatePicker(
    description='Fecha inicial:',
    value=datetime.now().date(),  # Establecer la fecha actual como valor inicial
    disabled=False
)

date_picker_fin = widgets.DatePicker(
    description='Fecha final:',
    value=datetime.now().date(),  # Establecer la fecha actual como valor inicial
    disabled=False
)

hbox_fecha= widgets.HBox([date_picker_inicio, date_picker_fin])

#..............................................................

opcion_area = widgets.RadioButtons(
    options=['Dibujar polígono en el mapa', 'Utilizar carta 1:50.000'],
    description='Seleccionar forma de determinar el área: ',
    layout=widgets.Layout(width='50%')
)






### Interfaz visual *
Se  debe modifcar la **fecha** y la **forma de selección del área** de interés para la clasificación.

In [ ]:
widgets.VBox([etiqueta_fecha,hbox_fecha, opcion_area])

En caso de elegir la opción Dibujar polígono en el mapa", con las herramientas de dibujo, se debe trazar el polígono.

Si se elije la opción "Utilizar carta 1:50.000" se debe seleccionar una a continuación. Tener en cuenta que al presionar alguna se va cargando en el mapa para visualizar.

In [ ]:
display(etiqueta, lista_registros)

Label(value='Seleccione una carta 1:50.000')

Select(description='Opciones', options=('2554-26-4', '2554-32-2', '2554-32-4', '2554-27-3', '2554-27-2', '2554…

## 4. Entrenamiento
Simplemente se debe ejecutar, no es necesario modificar nada en este punto, a menos que se desee cambiar algún parámetro del ENTRENAMIENTO del algoritmo, como son las muestras, región o período.

*Si se considera que el resultado obtenido de aplicar todo este algoritmo (sin modificar parámetros de esta solapa de entrenamiento) no genera un buen resultado, se pueden agregar puntos de muestreo de la zona en particular de interés (para ello se debe descomentar el código de la pestaña "Agregar muestras para un mejor resultado" y seguir los pasos)

### Muestras preestablecidas del entrenamiento

In [ ]:
# MUESTRAS
muestras = ee.FeatureCollection("projects/ee-gonzalodgign/assets/AguaTierra/Muestras_Argentina_Borrador")
#Map.addLayer(muestras, {}, 'muestras')

# ZONA DE ENTRENAMIENTO
zona_muestreo = ee.FeatureCollection("projects/ee-tenglerevelyn/assets/AREA_A-T")
#Map.addLayer(zona_muestreo, {}, 'zona_muestreo')

### Agregar muestras para un mejor resultado

Para agregar puntos de muestreo se debe subir a drive 3 archivos:

- Un shp con puntos de muestreo para categoría agua (debe tener una columna class con valor 1)

- Un shp con puntos de muestreo para la categoría no agua/tierra (debe tener una columna class con valor 2)

- Un shp de polígono cuya extención debe encerrar todos los nuevos puntos de muestreo anteriores.

Para que funcione se debe descomentar las líneas con un solo # del siguiente código.



In [ ]:
shp_path_agua = "ACÁ VA LA RUTA AL ARCHIVO SHP DE LOS PUNTOS DE MUESTREO DE LA CATEGORÍA AGUA EN DRIVE"

### Cargar archivo SHP como FeatureCollection utilizando geemap
muestras_extra_agua = geemap.shp_to_ee(shp_path_agua)

### Visualizar FeatureCollection en el mapa
Map.addLayer(muestras_extra_agua, {}, 'muestras_extra_A')

### Unir las muestras de agua nuevas con las preestablecidas
muestras.merge(muestras_extra_agua)

In [ ]:
shp_path_tierra = "ACÁ VA LA RUTA AL ARCHIVO SHP DE LOS PUNTOS DE MUESTREO DE LA CATEGORÍA NO AGUA/TIERRA EN DRIVE"

### Cargar archivo SHP como FeatureCollection utilizando geemap
muestras_extra_tierra = geemap.shp_to_ee(shp_path_tierra)

### Visualizar FeatureCollection en el mapa
Map.addLayer(muestras_extra_tierra, {}, 'muestras_extra_T')

### Unir las muestras de agua nuevas con las preestablecidas
muestras.merge(muestras_extra_tierra)

In [ ]:
shp_path_extencion = "ACÁ VA LA RUTA AL ARCHIVO SHP DEL POLÍGONO EN DRIVE"

### Cargar archivo SHP como FeatureCollection utilizando geemap
extencion_extra = geemap.shp_to_ee(shp_path_extencion)

### Visualizar FeatureCollection en el mapa
Map.addLayer(extencion_extra, {}, 'extencion_extra')

### Unir la extencion nueva con la preestablecida
zona_muestreo.merge(extencion_extra)

### Colección de imágenes del entrenamiento

Se puede modificar el período de entrenamiento.

In [ ]:
# Colección de imágenes Sentinel 2
s2 = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate('2023-01-01', '2023-03-31') \
    .filterBounds(zona_muestreo) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)

s2c_ = s2.reduce(ee.Reducer.mean())
s2c = s2c_.clip(zona_muestreo)

# Añadir capa al mapa
#Map.addLayer(s2c, imageVisParam2, 'S2')

### Indices espectrales

In [ ]:
# NDWI
NDWI = s2c.normalizedDifference(['B3_mean', 'B8_mean']).rename('NDWImean')

# Stack
stackIslas_a = NDWI

### Entrenamiento

In [ ]:
muestras_1 = muestras.randomColumn('random')

# División de muestras
entrenamiento = muestras_1.filter(ee.Filter.lt('random', 0.7))
validacion = muestras_1.filter(ee.Filter.gt('random', 0.7))

# Entrenar el clasificador utilizando datos de muestras
training = stackIslas_a.sampleRegions(
  collection=entrenamiento,
  properties=['class', 'random'],
  scale=30
)

# Bandas a utilizar en la clasificación
bandas_sel = ['NDWImean']

# Entrenar clasificador RF
classifier = ee.Classifier.minimumDistance().train(
    features = training,
    classProperty = 'class',
    inputProperties = bandas_sel
)

### Validación

In [ ]:
###Si se desea ver los resultados de la validación se debe descomentar las lineas con un solo # del siguiente código
### Clasifico la imagen
#classified_entrenamiento = stackIslas_a.classify(classifier)

### Muestrear las regiones
#test = classified_entrenamiento.sampleRegions(**{
#  'collection': validacion,
#  'properties': ['class'],
#  'scale': 30
#})

## Obtener la matriz de confusión
#testConfusionMatrix = test.errorMatrix('class', 'classification')

### Convertir la matriz de confusión a un DataFrame de pandas
#confusion_df = pd.DataFrame(testConfusionMatrix.getInfo())

### Imprimir la matriz de confusión como tabla
#print(confusion_df)

## 5. Clasificación
Una vez que se ejecuta este código, se **carga el resultado** en el mapa del punto 3. Tener en cuenta que mientras mas grande es la zona de análisis mas tardará el proceso.

### Funciones

In [ ]:
def obtenerFecha():
    # Obtener las fechas seleccionadas
    fecha_inicio = date_picker_inicio.value
    fecha_fin = date_picker_fin.value

    # Convertir las fechas al formato deseado ('YYYY-MM-DD')
    fecha_inicio_str = fecha_inicio.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')

    # Convertir las fechas en objetos ee.Date
    fecha_inicio_ee = ee.Date(fecha_inicio_str)
    fecha_fin_ee = ee.Date(fecha_fin_str)

    # Utilizar las fechas convertidas en el parámetro time_interval
    time_interval = (fecha_inicio_ee, fecha_fin_ee)

    # Retornar el resultado
    return time_interval

obtenerFecha()

(<ee.ee_date.Date at 0x7f321586f010>, <ee.ee_date.Date at 0x7f3215636bf0>)

### Estilo

In [ ]:
# Color de categorías Agua/NoAgua
N1Color = [
    '#0000ff',  # Corriente de agua - clase 1
    '#764C04'  # Tierra/NoAgua - clase 2
]

imageVisParam2 = {
    'bands': ["B3_mean", "B2_mean", "B1_mean"],
    'gamma': 1,
    'max': 2789.728888888889,
    'min': 1875.3822222222223,
    'opacity': 1
}


### Región a clasificar

In [ ]:
from zmq.constants import NULL
####....................................................................... formas de elegir un área de interés

selected_value = opcion_area.value
print("Valor seleccionado:", selected_value)

if (selected_value == 'Dibujar polígono en el mapa'):
  # ..................................................Región DIBUJADA
  Map.draw_features
  if (Map.draw_last_feature != None):
    extencion = ee.FeatureCollection(Map.draw_last_feature)#Roi Bounding Box (guarda en variable el bbox)
    #print("ok")
else:
  # ..................................................Región CARTA
  valor_carta = lista_registros.value
  geometria_seleccionada = cartas.filter(ee.Filter.eq("carac", valor_carta))
  extencion = ee.FeatureCollection(geometria_seleccionada)
####..............................................................................................................




# .................................................. Colección de imágenes Sentinel 2
s2_b = ee.ImageCollection("COPERNICUS/S2") \
    .filterDate(*obtenerFecha()) \
    .filterBounds(extencion) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)

s2c_b = s2_b.reduce(ee.Reducer.mean())
s2c_b = s2c_b.clip(extencion)

# Añadir capa al mapa
#Map.addLayer(s2c_b, imageVisParam2, 'S2')
#Map.centerObject(BOCA, 5)

# NDWI
NDWI_b = s2c_b.normalizedDifference(['B3_mean', 'B8_mean']).rename('NDWImean')

# Stack
stackIslas_b = NDWI_b


Valor seleccionado: Utilizar carta 1:50.000


In [ ]:
# Clasificar la imagen
classified_b = stackIslas_b.classify(classifier)

# Añadir capa clasificada al mapa
Map.addLayer(classified_b, {'min': 1, 'max': 2, 'palette': N1Color}, 'Clasificado')

## 6. Descarga Raster *

Este paso es **opcional**. Al ejecutarlo se genera un link para la descarga del archivo Raster de la clasificación (extender la solapa para verlo).

*Tener en cuenta que GEE tiene algunas limitaciones de procesamiento, por lo que puede lanzar errores cuando la región sea muy grande. Para ello es recomendable trabajar con regiones del tamaño de una carta 1:50.000 o menores

In [ ]:
url = classified_b.getDownloadURL({
    'name': 'LimiteAguaTierra',
    'scale': 10,
    'region': extencion.geometry()
})

print(url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a8966c1cff90b02e78056fd5a342677d-3d9871eadd6f43680921d81720a46183:getPixels


## 7. Vectorización y descarga *

Este paso es **opcional**.

El archivo vectorial resultante se **descarga en Drive**. En la pestaña "Descargar archivo vectroizado" se puede modificar la línea "descripcion = 'Linea_de_Agua'" con el nombre que tendrá el archivo. La descarga puede tardar varios minutos y una vez finalizado se debe buscar el archivo en nuestro Drive.

En el resultado final el agua se distingue por la columna label con valor 1 y no agua/tierra el valor 2.


*Tener en cuenta que GEE tiene algunas limitaciones de procesamiento, por lo que puede lanzar errores cuando la región sea muy grande. Para ello es recomendable trabajar con regiones del tamaño de una carta 1:50.000 o menores


### Vectorizar

In [ ]:
vectores = classified_b.reduceToVectors(
    reducer= ee.Reducer.countEvery(),
    scale=10,
    geometry = extencion
)

#print(vectores)
Map.addLayer(vectores, {}, 'Vectorizado')

### Descargar archivo vectorizado

In [ ]:
# Obtener la colección de entidades que deseas exportar
Agua = ee.FeatureCollection(vectores)

# Definir los parámetros de exportación
descripcion = 'Linea_de_Agua' #MODIFCAR
formato = 'SHP'  # Puedes cambiar el formato según tus necesidades

# Configurar la exportación
exportacion = ee.batch.Export.table.toDrive(
    collection=Agua,
    description=descripcion,
    folder='',
    fileNamePrefix=descripcion,
    fileFormat=formato
)

# Iniciar la exportación
exportacion.start()

### Correr la siguiente línea las veces que sea necesario hasta que el estado sea COMPLETED, que será cuando se haya descargado el resultado en nuestro Drive. Tener en cuenta que puede tardar unos minutos.

In [ ]:
status = exportacion.status()
print("Estado de la exportación:", status["state"])

Estado de la exportación: COMPLETED


## 8. Tratamiento al archivo vectorial

Una vez descargado el vector, se le puede realizar una serie de procesos para un mejor resultado. Para ello se pueden utilizar las herramientas de QGis de simplificado y suavizado.